## Install Ktor

- ktor-client-core
- ktor-client-cio

In [1]:
USE {
    val ktorVersion = "3.1.0"

    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("io.ktor:ktor-client-core:$ktorVersion")
        implementation("io.ktor:ktor-client-cio:$ktorVersion")
    }
}

In [2]:
%use ktor-client
%use dataframe

In [4]:
import io.ktor.client.*
import io.ktor.client.engine.cio.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import kotlinx.coroutines.*
import java.time.*
import java.math.BigDecimal
import java.time.format.DateTimeFormatter

val client = HttpClient(CIO)

@Serializable
data class ApiResponse(
    val code: String,
    val msg: String,
    val requestTime: Long,
    val data: List<List<String>>,
)

data class CandleRecord(
    val timestamp: Long,
    val open: BigDecimal,
    val high: BigDecimal,
    val low: BigDecimal,
    val close: BigDecimal,
    val volume: BigDecimal,
    val quote_currency_volume: BigDecimal,
)

suspend fun fetchCandles(
    symbol: String = "BTCUSDT",
    granularity: String = "1m",
    startDate: String = LocalDate.now().minusDays(1).format(DateTimeFormatter.ISO_DATE),
    endDate: String = LocalDate.now().format(DateTimeFormatter.ISO_DATE),
): List<CandleRecord> {
    require(startDate < endDate) { "startDate must be earlier than endDate" }
    require(granularity in setOf("1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1M")) { "granularity must be one of 1m, 5m, 15m, 30m, 1h, 4h, 1d, 1w, 1M" }
    require(symbol.isNotBlank()) { "symbol must not be blank" }
    
    suspend fun fetchCandlesChunk(
        symbol: String,
        granularity: String,
        startTime: Long,
        endTime: Long,
    ): List<CandleRecord> = """
      https://api.bitget.com/api/v2/mix/market/history-candles?
      symbol=$symbol&
      granularity=$granularity&
      limit=200&
      productType=usdt-futures&
      startTime=$startTime&
      endTime=$endTime
      """.trimIndent()
          .let { client.get(it).readBytes().decodeToString() }
          .let { Json.decodeFromString<ApiResponse>(it) }
          .data.map { row ->
              CandleRecord(
                  timestamp = row[0].toLong(),
                  open = row[1].toBigDecimal(),
                  high = row[2].toBigDecimal(),
                  low = row[3].toBigDecimal(),
                  close = row[4].toBigDecimal(),
                  volume = row[5].toBigDecimal(),
                  quote_currency_volume = row[6].toBigDecimal(),
              )
          }
    
    val startMillis = LocalDate.parse(startDate).atStartOfDay().toInstant(ZoneOffset.UTC).toEpochMilli()
    val endMillis = LocalDate.parse(endDate).plusDays(1).atStartOfDay().toInstant(ZoneOffset.UTC).toEpochMilli()

    val results = mutableListOf<CandleRecord>()
    var currentStart = startMillis

    while (currentStart < endMillis) {
        // 현재 구간 호출: startTime = currentStart, endTime = 전역 endMillis
        val chunk = fetchCandlesChunk(
            symbol = symbol,
            granularity = granularity,
            startTime = currentStart,
            endTime = endMillis,
        )

        if (chunk.isEmpty()) break

        results.addAll(chunk)

        // 반환된 건수가 limit 미만이면 더 이상 데이터가 없다고 가정
        if (chunk.size < 200) break

        // 다음 호출을 위한 시작시간: 마지막 캔들의 timestamp + 1ms
        val lastTimestamp = chunk.last().timestamp
        if (lastTimestamp < currentStart) break  // 무한 루프 방지
        currentStart = lastTimestamp + 1
    }
    
    return results
}

In [17]:
import java.time.*
import java.time.Instant

val startDate = "2025-02-11"
val endDate = "2025-02-12"

val df = runBlocking {
    fetchCandles(
        symbol = "BTCUSDT",
        granularity = "1m",
        startDate = startDate,
        endDate = endDate
    )
        .toDataFrame<CandleRecord>()
        .let {
            it.insert("time") {
                Instant.ofEpochMilli("timestamp"<Long>())
                    .atZone(ZoneOffset.UTC).toLocalDateTime()
            }.after("timestamp")
        }
}

In [20]:
df.writeCSV("candles_1m_${startDate}_$endDate.csv")